In [ ]:
%pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.4 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
from datasets import load_dataset

dataset_laptop = load_dataset("jakartaresearch/semeval-absa", name='laptop')
dataset_restaurant = load_dataset("jakartaresearch/semeval-absa", name='restaurant')
dataset_fabsa = load_dataset("jordiclive/FABSA")

Generating train split:   0%|          | 0/3048 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/3044 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/800 [00:00<?, ? examples/s]

# Deberta for ATSC

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the ABSA model and tokenizer
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device='cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

## Evaluate on SemEval on the domain: laptops

In [ ]:
from tqdm.auto import tqdm

labels = []
predictions = []

POLARITY_SET = set(['positive', 'negative', 'neutral'])

for i, data in enumerate(tqdm(dataset_laptop['validation'])):
    terms = data['aspects']['term']
    polarities = data['aspects']['polarity']

    for term, polarity in zip(terms, polarities):
        if polarity not in POLARITY_SET or term == '':
            continue

        prediction = classifier(data['text'], text_pair=term)
        predictions.append(prediction[0]['label'].lower())
        labels.append(polarity)


print(classification_report(labels, predictions))
print(f"accuracy_score = {accuracy_score(labels, predictions)}")

  0%|          | 0/800 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       0.69      0.94      0.80       128
     neutral       0.78      0.57      0.66       169
    positive       0.91      0.91      0.91       341

    accuracy                           0.83       638
   macro avg       0.79      0.81      0.79       638
weighted avg       0.83      0.83      0.82       638

accuracy_score = 0.8275862068965517


## Evaluate on SemEval on the domain: restaurant

In [ ]:
from tqdm.auto import tqdm

labels = []
predictions = []

POLARITY_SET = set(['positive', 'negative', 'neutral'])

for i, data in enumerate(tqdm(dataset_restaurant['validation'])):
    terms = data['aspects']['term']
    polarities = data['aspects']['polarity']

    for term, polarity in zip(terms, polarities):
        if polarity not in POLARITY_SET or term == '':
            continue

        prediction = classifier(data['text'], text_pair=term)
        predictions.append(prediction[0]['label'].lower())
        labels.append(polarity)


print(classification_report(labels, predictions))
print(f"accuracy_score = {accuracy_score(labels, predictions)}")

  0%|          | 0/800 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       0.85      0.80      0.82       196
     neutral       0.56      0.78      0.66       196
    positive       0.94      0.86      0.90       728

    accuracy                           0.84      1120
   macro avg       0.79      0.81      0.79      1120
weighted avg       0.86      0.84      0.84      1120

accuracy_score = 0.8366071428571429


In [ ]:
set(predictions), set(labels), set(predictions) == set(labels)

{'', 'negative', 'neutral', 'positive'}

## Evaluate on FABSA

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import classification_report, accuracy_score


labels = []
predictions = []

POLARITY_SET = set(['positive', 'negative', 'neutral'])

for i, data in enumerate(tqdm(dataset_fabsa['test'])):
    for term, polarity in data['labels']:
        if polarity not in POLARITY_SET or term == '':
            continue

        prediction = classifier(data['text'], text_pair=term)
        predictions.append(prediction[0]['label'].lower())
        labels.append(polarity)

print(classification_report(labels, predictions))
print(f"accuracy_score = {accuracy_score(labels, predictions)}")

  0%|          | 0/1587 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       0.95      0.67      0.79       896
     neutral       0.32      0.73      0.45        91
    positive       0.88      0.95      0.92      1825

    accuracy                           0.86      2812
   macro avg       0.72      0.78      0.72      2812
weighted avg       0.89      0.86      0.86      2812

accuracy_score = 0.8577524893314367


In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import classification_report, accuracy_score


labels = []
predictions = []

POLARITY_SET = set(['positive', 'negative', 'neutral'])

for i, data in enumerate(tqdm(dataset_fabsa['test'])):
    for term, polarity in data['labels']:
        if polarity not in POLARITY_SET or term == '':
            continue

        prediction = classifier(data['text'], text_pair=term.split(': ')[-1])
        predictions.append(prediction[0]['label'].lower())
        labels.append(polarity)

print(classification_report(labels, predictions))
print(f"accuracy_score = {accuracy_score(labels, predictions)}")

  0%|          | 0/1587 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       0.94      0.72      0.82       896
     neutral       0.34      0.69      0.46        91
    positive       0.90      0.95      0.92      1825

    accuracy                           0.87      2812
   macro avg       0.73      0.79      0.73      2812
weighted avg       0.89      0.87      0.88      2812

accuracy_score = 0.8716216216216216


## Sanity Check

In [ ]:
text = """Apart from a few "buffering moments" ordering our shopping has never been easier"""
{
    aspect: classifier(text, text_pair=aspect)[0]['label']
    for aspect in (
        [
            "speed",
            "velocity",
            "waiter",
            "weather",
            "shoes",
            "United States",
            "North Korea",
            "Iran",
            "United Kingdom",
        ]
    )
}

{'speed': 'Positive',
 'velocity': 'Positive',
 'waiter': 'Positive',
 'weather': 'Positive',
 'shoes': 'Positive',
 'United States': 'Positive',
 'North Korea': 'Negative',
 'Iran': 'Positive',
 'United Kingdom': 'Positive'}

# InstructABSA for ATSC
It uses a trained Tk-base-pos-def model

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

classifier = pipeline("text2text-generation", model="kevinscaria/atsc_tk-instruct-base-def-pos-neg-neut-combined", device='cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
bos_instruction = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'.
        Otherwise, the output should be 'neutral'. For aspects which are classified as noaspectterm, the sentiment is none.
        Positive example 1-
        input: With the great variety on the menu , I eat here often and never get bored. The aspect is menu.
        output: positive
        Positive example 2-
        input: Great food, good size menu, great service and an unpretensious setting. The aspect is food.
        output: positive
        Negative example 1-
        input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it. The aspect is toast.
        output: negative
        Negative example 2-
        input: The seats are uncomfortable if you are sitting against the wall on wooden benches. The aspect is seats.
        output: negative
        Neutral example 1-
        input: I asked for seltzer with lime, no ice. The aspect is seltzer with lime.
        output: neutral
        Neutral example 2-
        input: They wouldnt even let me finish my glass of wine before offering another. The aspect is glass of wine.
        output: neutral
        Now complete the following example-
        input: """

sep_instruction = ' The aspect is '
eos_instruction = '.\noutput:'

text = "Boot time is super fast, around anywhere from 35 seconds to 1 minute."
aspect = "Boot time"

input_text = bos_instruction + text + sep_instruction + aspect + eos_instruction

classifier(input_text, max_new_tokens=1)[0]['generated_text']

'positive'

## Sanity Check

In [ ]:
text = "Apart from a few \"buffering moments\" ordering our shopping has never been easier"

{
    aspect: classifier(bos_instruction + text + sep_instruction + aspect + eos_instruction, max_new_tokens=1)[0]['generated_text']
    for aspect in (
        [
            "speed",
            "velocity",
            "waiter",
            "weather",
            "shoes",
            "North Korea",
            "United States",
            "Iran"
        ]
    )
}

{'speed': 'positive',
 'velocity': 'positive',
 'waiter': 'positive',
 'weather': 'negative',
 'shoes': 'positive',
 'North Korea': 'negative',
 'United States': 'neutral',
 'Iran': 'positive'}

## Evaluate on SemEval on the domain: laptop

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import classification_report, accuracy_score

labels = []
predictions = []

for i, data in enumerate(tqdm(dataset_laptop['validation'])):
    for term, polarity in zip(data['aspects']['term'], data['aspects']['polarity']):
        if polarity == 'conflict':
            continue
        if term == '':
            pass
            # predictions.append('')
        else:
            aspect = term
            text = data['text']
            input_text = bos_instruction + text + sep_instruction + aspect + eos_instruction
            prediction = classifier(input_text, max_new_tokens=1)
            predictions.append(prediction[0]['generated_text'].lower())
            labels.append(polarity)



print(classification_report(labels, predictions))
print(f"accuracy_score = {accuracy_score(labels, predictions)}")

  0%|          | 0/800 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       0.67      0.88      0.76       128
     neutral       0.78      0.57      0.66       169
    positive       0.89      0.91      0.90       341

    accuracy                           0.82       638
   macro avg       0.78      0.79      0.78       638
weighted avg       0.82      0.82      0.81       638

accuracy_score = 0.8150470219435737


## Evaluate on SemEval on the domain: restaurant

In [ ]:
from tqdm.auto import tqdm

labels = []
predictions = []

for i, data in enumerate(tqdm(dataset_restaurant['validation'])):
    for term, polarity in zip(data['aspects']['term'], data['aspects']['polarity']):
        if polarity == 'conflict':
            continue
        if term == '':
            pass
            # predictions.append('')
        else:
            aspect = term
            text = data['text']
            input_text = bos_instruction + text + sep_instruction + aspect + eos_instruction
            prediction = classifier(input_text, max_new_tokens=1)
            predictions.append(prediction[0]['generated_text'].lower())
            labels.append(polarity)


from sklearn.metrics import classification_report, accuracy_score

print(classification_report(labels, predictions))
print(f"accuracy_score = {accuracy_score(labels, predictions)}")

  0%|          | 0/800 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       0.80      0.88      0.84       196
     neutral       0.77      0.43      0.55       196
    positive       0.89      0.97      0.93       728

    accuracy                           0.86      1120
   macro avg       0.82      0.76      0.77      1120
weighted avg       0.85      0.86      0.85      1120

accuracy_score = 0.8607142857142858


In [ ]:
set(predictions), set(labels), set(predictions) == set(labels)

({'negative', 'neutral', 'positive'},
 {'negative', 'neutral', 'positive'},
 True)

## Evaluate on FABSA

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import classification_report, accuracy_score


labels = []
predictions = []

POLARITY_SET = set(['positive', 'negative', 'neutral'])

for i, data in enumerate(tqdm(dataset_fabsa['test'])):
    for term, polarity in data['labels']:
        if polarity not in POLARITY_SET or term == '':
            continue

        input_text = (
            bos_instruction
            + data['text']
            + sep_instruction
            + aspect
            + eos_instruction
        )
        prediction = classifier(input_text, max_new_tokens=1)
        predictions.append(prediction[0]['generated_text'].lower())
        labels.append(polarity)

print(classification_report(labels, predictions))
print(f"accuracy_score = {accuracy_score(labels, predictions)}")

  0%|          | 0/1587 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (601 > 512). Running this sequence through the model will result in indexing errors


              precision    recall  f1-score   support

    negative       0.83      0.83      0.83       896
     neutral       0.20      0.85      0.33        91
    positive       0.98      0.83      0.90      1825

    accuracy                           0.83      2812
   macro avg       0.67      0.84      0.69      2812
weighted avg       0.91      0.83      0.86      2812

accuracy_score = 0.8303698435277382


In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import classification_report, accuracy_score


labels = []
predictions = []

POLARITY_SET = set(['positive', 'negative', 'neutral'])

for i, data in enumerate(tqdm(dataset_fabsa['test'])):
    for term, polarity in data['labels']:
        if polarity not in POLARITY_SET or term == '':
            continue

        input_text = (
            bos_instruction
            + data['text']
            + sep_instruction
            + aspect.split(": ")[-1]
            + eos_instruction
        )
        prediction = classifier(input_text, max_new_tokens=1)
        predictions.append(prediction[0]['generated_text'].lower())
        labels.append(polarity)

print(classification_report(labels, predictions))
print(f"accuracy_score = {accuracy_score(labels, predictions)}")

  0%|          | 0/1587 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       0.83      0.83      0.83       896
     neutral       0.20      0.85      0.33        91
    positive       0.98      0.83      0.90      1825

    accuracy                           0.83      2812
   macro avg       0.67      0.84      0.69      2812
weighted avg       0.91      0.83      0.86      2812

accuracy_score = 0.8303698435277382


# PyABSA

In [ ]:
%pip install -qU pyabsa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
from pyabsa import AspectTermExtraction as ATEPC


aspect_extractor = ATEPC.AspectExtractor('multilingual', auto_device=True)

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen impo

[2024-05-20 22:58:12] (2.4.1.post1) PyABSA(2.4.1.post1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


[2024-05-20 22:58:25] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-05-20 22:58:25] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-05-20 22:58:25] (2.4.1.post1) Downloading checkpoint:multilingual 
[2024-05-20 22:58:25] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets


Find zipped checkpoint: ./checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc_Multilingual_cdw_apcacc_85.1_apcf1_80.2_atef1_76.45.zip, unzipping


...

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
instances = [
    # "The battery life of this phone is incredible, but the screen cracks too easily and the camera quality is disappointing.",
    # "While the hotel's location is perfect and the breakfast delicious, the room was noisy and the staff unhelpful.",
    "Apart from a few \"buffering moments\" ordering our shopping has never been easier"
]

# instance inference
atepc_result = aspect_extractor.predict(instances,
    save_result=True,
    print_result=True,  # print the result
    ignore_error=True,  # ignore the error when the model cannot predict the input
)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


[2024-05-20 23:03:56] (2.4.1.post1) The results of aspect term extraction have been saved in /content/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2024-05-20 23:03:56] (2.4.1.post1) Example 0: Apart from a few " buffering moments " ordering our <shopping:Negative Confidence:0.4708> has never been easier


In [ ]:
from pprint import pprint

pprint(atepc_result, width=400)

[{'IOB': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ASP', 'O', 'O', 'O', 'O'],
  'aspect': ['shopping'],
  'confidence': [0.4708],
  'position': [[10]],
  'probs': [[0.47077029943466187, 0.39421144127845764, 0.1350182294845581]],
  'sentence': 'Apart from a few " buffering moments " ordering our shopping has never been easier',
  'sentiment': ['Negative'],
  'tokens': ['Apart', 'from', 'a', 'few', '"', 'buffering', 'moments', '"', 'ordering', 'our', 'shopping', 'has', 'never', 'been', 'easier']}]
